In [1]:
import torchvision

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content/drive/MyDrive/sejong/far'

In [2]:
%cd '/content/drive/MyDrive/sejong/far'

/content/drive/MyDrive/sejong/far


# Requirements

In [3]:
import h5py
import random
import os
import os.path
import shutil
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2
import json
import glob
import random
from PIL import Image

from sklearn import preprocessing

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

import torch.utils.data as data_utl
from torch.utils.data.dataloader import default_collate

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import SequentialSampler

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
# 1차 데이터셋
img_list_1 = sorted(glob.glob('./data/data_1/detection/images/*.jpg'), key=lambda x: os.path.splitext(x)[0])
label_list_1 = sorted(glob.glob('./data/data_1/detection/labels/*.txt'), key=lambda x: os.path.splitext(x)[0])

# 2차 데이터셋
img_list_2 = sorted(glob.glob('./data/data_2/images/*.jpg'), key=lambda x: os.path.splitext(x)[0])
label_list_2 = sorted(glob.glob('./data/data_2/labels/*.txt'), key=lambda x: os.path.splitext(x)[0])

In [ ]:
# for idx,img in enumerate(img_list_1):
#   os.rename(img, f'./data/data_1/detection/images/{idx}.jpg')
# for idx,label in enumerate(label_list_1):
#   os.rename(label, f'./data/data_1/detection/labels/{idx}.txt')

# for idx,img in enumerate(img_list_2):
#   os.rename(img, f'./data/data_2/images/{idx+794}.jpg')
# for idx,label in enumerate(label_list_2):
#   os.rename(label, f'./data/data_2/labels/{idx+794}.txt')

FileNotFoundError: [Errno 2] No such file or directory: './data/data_1/detection/labels/0001-20230612-081810.txt' -> './data/data_1/detection/labels/0.txt'

In [ ]:
# 1차 데이터셋
img_list_1 = sorted(glob.glob('./data/data_1/detection/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
label_list_1 = sorted(glob.glob('./data/data_1/detection/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# 2차 데이터셋
img_list_2 = sorted(glob.glob('./data/data_2/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
label_list_2 = sorted(glob.glob('./data/data_2/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

print(len(img_list_1))
print(len(label_list_1))
print(len(img_list_2))
print(len(label_list_2))

3924
3924
0
0


In [ ]:
for img in img_list_2:
  shutil.move(img,'./data/data_1/detection/images/')
for label in label_list_2:
  shutil.move(label,'./data/data_1/detection/labels/')

# 라벨 재정의
---
* 1차 데이터셋: 797개
* 2차 데이터셋: 3130개  
> 총 데이터셋: 3927개  
> 라벨 재정의 이후: 2918개

## 클래스 축소
---
> 3 -> 0  
> 4 -> 1  
> 5 -> 2

In [ ]:
def modify_class(label):
    if label == '3':
        return '0'
    elif label == '4':
        return '1'
    elif label == '5':
        return '2'
    return label

In [ ]:
# label_list = sorted(glob.glob('./data/data_1/detection/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# for label in label_list:
#   with open (label,'r') as f:
#       lines = f.readlines()
#   # 클래스 값 수정
#   modified_lines = []
#   for line in lines:
#       parts = line.strip().split(' ')  # 각 line을 공백 기준으로 나눔
#       parts[0] = modify_class(parts[0])  # 클래스 값 수정
#       modified_lines.append(' '.join(parts) + '\n')  # 수정된 내용을 다시 합침
#   # 파일 덮어쓰기
#   with open(label, 'w') as f:
#       f.writelines(modified_lines)

#   print(f"Modified {label}")

Modified ['0 0.346755 0.901659 0.020318 0.079874\n', '2 0.412288 0.520604 0.014595 0.060541\n', '2 0.446200 0.470492 0.016026 0.060033\n', '2 0.440047 0.445309 0.012878 0.058506\n', '2 0.549794 0.210267 0.012592 0.037139\n', '2 0.571400 0.172365 0.007727 0.028490\n', '2 0.688301 0.580128 0.012019 0.057489\n']
Modified ['1 0.608745 0.192206 0.010875 0.040700\n', '0 0.377518 0.923535 0.016026 0.077839\n', '2 0.438044 0.487027 0.012878 0.054436\n', '2 0.415722 0.503816 0.016312 0.052401\n', '2 0.452209 0.429029 0.013164 0.057489\n', '2 0.688731 0.580637 0.014022 0.053419\n', '2 0.553800 0.200600 0.010302 0.039174\n']
Modified ['1 0.646234 0.190171 0.010875 0.038665\n', '1 0.623769 0.191188 0.010016 0.040700\n', '2 0.447487 0.414276 0.015739 0.053419\n', '2 0.422447 0.485501 0.013164 0.054436\n', '2 0.432320 0.502544 0.014595 0.056980\n', '0 0.417726 0.929385 0.020318 0.081400\n', '2 0.689017 0.580891 0.013450 0.048840\n']
Modified ['0 0.363782 0.902422 0.020604 0.084453\n', '0 0.540064 0.

## 클래스2 삭제
---
* 인도 보행자

In [ ]:
# label_list = sorted(glob.glob('./data/data_1/detection/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# for label in label_list:
#   with open (label,'r') as f:
#       lines = f.readlines()

#   # 클래스가 2가 아닌 줄만 필터링
#   modified_lines = [line for line in lines if line.strip().split(' ')[0] != '2']

#   # 파일 덮어쓰기
#   with open(label, 'w') as f:
#       f.writelines(modified_lines)

#   print(f"Modified {label}")

Modified ./data/data_1/detection/labels/0.txt
Modified ./data/data_1/detection/labels/1.txt
Modified ./data/data_1/detection/labels/2.txt
Modified ./data/data_1/detection/labels/3.txt
Modified ./data/data_1/detection/labels/4.txt
Modified ./data/data_1/detection/labels/5.txt
Modified ./data/data_1/detection/labels/6.txt
Modified ./data/data_1/detection/labels/7.txt
Modified ./data/data_1/detection/labels/8.txt
Modified ./data/data_1/detection/labels/9.txt
Modified ./data/data_1/detection/labels/10.txt
Modified ./data/data_1/detection/labels/11.txt
Modified ./data/data_1/detection/labels/12.txt
Modified ./data/data_1/detection/labels/13.txt
Modified ./data/data_1/detection/labels/14.txt
Modified ./data/data_1/detection/labels/15.txt
Modified ./data/data_1/detection/labels/16.txt
Modified ./data/data_1/detection/labels/17.txt
Modified ./data/data_1/detection/labels/18.txt
Modified ./data/data_1/detection/labels/19.txt
Modified ./data/data_1/detection/labels/20.txt
Modified ./data/data_1/

## 클래스 2만 존재해서 background가 된 데이터셋 삭제

In [ ]:
label_list = sorted(glob.glob('./data/data_1/detection/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

empty_files = []
for label in label_list:
  with open (label,'r') as f:
      lines = f.readlines()
      # lines가 빈 리스트인 경우
      if not lines:
          empty_files.append(label)

empty_files

['./data/data_1/detection/labels/1.txt',
 './data/data_1/detection/labels/7.txt',
 './data/data_1/detection/labels/29.txt',
 './data/data_1/detection/labels/30.txt',
 './data/data_1/detection/labels/71.txt',
 './data/data_1/detection/labels/80.txt',
 './data/data_1/detection/labels/89.txt',
 './data/data_1/detection/labels/90.txt',
 './data/data_1/detection/labels/128.txt',
 './data/data_1/detection/labels/134.txt',
 './data/data_1/detection/labels/143.txt',
 './data/data_1/detection/labels/146.txt',
 './data/data_1/detection/labels/150.txt',
 './data/data_1/detection/labels/171.txt',
 './data/data_1/detection/labels/188.txt',
 './data/data_1/detection/labels/194.txt',
 './data/data_1/detection/labels/195.txt',
 './data/data_1/detection/labels/199.txt',
 './data/data_1/detection/labels/200.txt',
 './data/data_1/detection/labels/202.txt',
 './data/data_1/detection/labels/208.txt',
 './data/data_1/detection/labels/209.txt',
 './data/data_1/detection/labels/212.txt',
 './data/data_1/detec

In [ ]:
# # background 이미지 삭제
# for files in empty_files:
#   img = files.replace('labels','images').replace('txt','jpg')
#   os.remove(img)
# # background 라벨 삭제
# for files in empty_files:
#   os.remove(files)

In [ ]:
# # 이름 재정의
# img_list = sorted(glob.glob('./data/data_1/detection/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
# label_list = sorted(glob.glob('./data/data_1/detection/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# for idx,img in enumerate(img_list):
#   os.rename(img, f'./data/data_1/detection/images/{idx}.jpg')
# for idx,label in enumerate(label_list):
#   os.rename(label, f'./data/data_1/detection/labels/{idx}.txt')

# 라벨 비율 확인
---
* jaywalker: 891장
* pedestrian: 2027장

In [33]:
split_ratio=[9,0.8,0.2]
modes = ['train', 'val', 'test']
det_data_dir = './data/detection'
# det_data_dir = './data/new_data'

for data_dir, img_ext, label_ext in [(det_data_dir, '.jpg', '.txt')]:
    # img_list = sorted(glob.glob(f'{data_dir}/images/*{img_ext}'), key=lambda x: os.path.splitext(x)[0])
    jaywalker = []
    pedestrian = []
    sidewalk = []

    label_path = glob.glob(f'{det_data_dir}/labels/*{label_ext}')
    # label_path = sorted(glob.glob('./ori_seg/train/labels/*.txt'),key=lambda x: int(x.split('/')[-1].split('.')[0]))[:831]
    for label in label_path:
        with open(label, 'r') as f:
            lines = f.readlines()
            jaywalker_coords = []  # jaywalker의 좌표를 저장할 리스트
            pedestrian_coords = []  # pedestrian의 좌표를 저장할 리스트
            for line in lines:
                if line.strip() and line[0] == '0':  # 클래스 ID가 '0'인 경우
                    jaywalker_coords.append(line.strip())  # 해당 라인을 좌표 리스트에 추가
                elif line.strip() and line[0] == '1':  # 클래스 ID가 '1'인 경우
                    pedestrian_coords.append(line.strip())  # 해당 라인을 좌표 리스트에 추가
            # 파일에 '0'과 '1'이 모두 있는 경우 jaywalker에 추가
            if jaywalker_coords and pedestrian_coords:
                jaywalker.append([label.split('/')[-1], jaywalker_coords + pedestrian_coords])
            # 파일에 '0'만 있는 경우에도 jaywalker에 추가
            elif jaywalker_coords:
                jaywalker.append([label.split('/')[-1], jaywalker_coords])
            # 파일에 '1'만 있는 경우 pedestrian에 추가
            elif pedestrian_coords:
                pedestrian.append([label.split('/')[-1], pedestrian_coords])

    random.seed(100)
    random.shuffle(jaywalker)
    random.shuffle(pedestrian)
    # jaywalker = sorted(jaywalker, key=lambda x: x[0])
    # pedestrian = sorted(pedestrian, key=lambda x: x[0])
    # Calculate the split sizes
    data_num = len(label_path)
    val_num = int(np.floor(data_num * (split_ratio[1] / 10)))
    test_num = int(np.floor(data_num * (split_ratio[2] / 10)))
    train_num = data_num - (val_num + test_num) # Adjust to avoid mismatch

    # Move to Directory
    ## Test: jaywalker:30, pedestrian:30
    for i in range(test_num//2):
      shutil.copy(f'{det_data_dir}/labels/{jaywalker[i][0]}',f'./data/new_data/{modes[2]}/labels/')
      shutil.copy(f'{det_data_dir}/labels/{pedestrian[i][0]}',f'./data/new_data/{modes[2]}/labels/')
      shutil.copy(f"{det_data_dir}/images/{jaywalker[i][0].replace('.txt','.jpg')}",f"./data/new_data/{modes[2]}/images/")
      shutil.copy(f"{det_data_dir}/images/{pedestrian[i][0].replace('.txt','.jpg')}",f"./data/new_data/{modes[2]}/images/")

    ## Val: jaywalker:30, pedestrian:30
    for i in range(test_num//2,(test_num//2) + (val_num//2)):
      shutil.copy(f'{det_data_dir}/labels/{jaywalker[i][0]}',f'./data/new_data/{modes[1]}/labels/')
      shutil.copy(f'{det_data_dir}/labels/{pedestrian[i][0]}',f'./data/new_data/{modes[1]}/labels/')
      shutil.copy(f"{det_data_dir}/images/{jaywalker[i][0].replace('.txt','.jpg')}",f"./data/new_data/{modes[1]}/images/")
      shutil.copy(f"{det_data_dir}/images/{pedestrian[i][0].replace('.txt','.jpg')}",f"./data/new_data/{modes[1]}/images/")

    ## Train: jaywalker:11, pedestrian:423
    for i in range((test_num//2) + (val_num//2),len(jaywalker)):
      shutil.copy(f'{det_data_dir}/labels/{jaywalker[i][0]}',f'./data/new_data/{modes[0]}/labels/')
      shutil.copy(f"{det_data_dir}/images/{jaywalker[i][0].replace('.txt','.jpg')}",f"./data/new_data/{modes[0]}/images/")
    for i in range((test_num//2) + (val_num//2),len(pedestrian)):
      shutil.copy(f'{det_data_dir}/labels/{pedestrian[i][0]}',f'./data/new_data/{modes[0]}/labels/')
      shutil.copy(f"{det_data_dir}/images/{pedestrian[i][0].replace('.txt','.jpg')}",f"./data/new_data/{modes[0]}/images/")

    print('val',val_num)
    print('test',test_num)
    print('train',train_num)

print('jaywalker',len(jaywalker))
print('pedestrian',len(pedestrian))
print(len(glob.glob('./data/detection/images/*.jpg')),len(glob.glob('./data/detection/labels/*.txt')))

val 233
test 58
train 2627
jaywalker 891
pedestrian 2027
2918 2918


In [38]:
# # img = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/detection/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
# # label = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/detection/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
# seg_img = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/modified_seg/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
# seg_label = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/modified_seg/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# # for i in img:
# #   shutil.copy(i,'/content/drive/MyDrive/sejong/far/data/new_data/images/')
# # for i in label:
# #   shutil.copy(i,'/content/drive/MyDrive/sejong/far/data/new_data/labels/')
# for idx,i in enumerate(seg_img):
#   shutil.copy(i,f'/content/drive/MyDrive/sejong/far/data/new_data/train/images/{idx+2918}.jpg')
# for idx,i in enumerate(seg_label):
#   shutil.copy(i,f'/content/drive/MyDrive/sejong/far/data/new_data/train/labels/{idx+2918}.txt')

In [41]:
train_img = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/train/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
train_label = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/train/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# val_img = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/val/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
# val_label = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/val/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

# test_img = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/test/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
# test_label = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/test/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

for idx,img in enumerate(train_img):
  os.rename(img,f'./data/new_data/train/images/{idx}.jpg')
for idx,label in enumerate(train_label):
  os.rename(label,f'./data/new_data/train/labels/{idx}.txt')

# for idx,img in enumerate(val_img):
#   os.rename(img,f'./data/new_data/val/images/{idx}.jpg')
# for idx,label in enumerate(val_label):
#   os.rename(label,f'./data/new_data/val/labels/{idx}.txt')

# for idx,img in enumerate(test_img):
#   os.rename(img,f'./data/new_data/test/images/{idx}.jpg')
# for idx,label in enumerate(test_label):
#   os.rename(label,f'./data/new_data/test/labels/{idx}.txt')

In [42]:
train_img = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/train/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
train_label = sorted(glob.glob('/content/drive/MyDrive/sejong/far/data/new_data/train/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
print(train_img[-1])
print(train_label[-1])
print(len(train_img))
print(len(train_label))

/content/drive/MyDrive/sejong/far/data/new_data/train/images/5049.jpg
/content/drive/MyDrive/sejong/far/data/new_data/train/labels/5049.txt
5050
5050


In [18]:
with open('./data/modified_seg/labels/1000.txt','r') as f:
  lines = f.readlines()
  print(lines)

['1 0.3640625 0.4640625 0.01796875 0.0890625']


In [26]:
import cv2
from google.colab.patches import cv2_imshow

def draw_bounding_box(image_path, bbox):
    img = cv2.imread(image_path)
    # img = crop(img)
    height, width, _ = img.shape
    x_center, y_center, box_width, box_height = bbox

    x1 = int((x_center - box_width / 2) * width)
    y1 = int((y_center - box_height / 2) * height)
    x2 = int((x_center + box_width / 2) * width)
    y2 = int((y_center + box_height / 2) * height)

    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2_imshow(img)

image_path = './data/new_data/train/images/2627.jpg'

bboxes = [0.3640625, 0.4640625, 0.01796875, 0.0890625]


# for bbox in bboxes:
#   draw_bounding_box(image_path, bbox)

draw_bounding_box(image_path, bboxes)


Output hidden; open in https://colab.research.google.com to view.

In [30]:
# 1차 데이터셋
img_list = sorted(glob.glob('./data/modified_seg/images/*.jpg'), key=lambda x: int(x.split('/')[-1].split('.')[0]))
label_list = sorted(glob.glob('./data/modified_seg/labels/*.txt'), key=lambda x: int(x.split('/')[-1].split('.')[0]))

print(img_list[-1])
print(label_list[-1])
print(len(img_list))
print(len(label_list))

./data/modified_seg/images/2421.jpg
./data/modified_seg/labels/2421.txt
2422
2422


In [32]:
len(glob.glob('./data/new_data/train/labels/*.txt'))

5050

In [58]:
split_ratio=[9,0.8,0.2]
modes = ['train', 'val', 'test']
# det_data_dir = './data/detection'
det_data_dir = './data/new_data'

for data_dir, img_ext, label_ext in [(det_data_dir, '.jpg', '.txt')]:
    # img_list = sorted(glob.glob(f'{data_dir}/images/*{img_ext}'), key=lambda x: os.path.splitext(x)[0])
    jaywalker = []
    pedestrian = []
    sidewalk = []

    label_path = sorted(glob.glob(f'{det_data_dir}/train/labels/*{label_ext}'),key=lambda x: int(x.split('/')[-1].split('.')[0]))
    # label_path = sorted(glob.glob('./ori_seg/train/labels/*.txt'),key=lambda x: int(x.split('/')[-1].split('.')[0]))[:831]
    for label in label_path:
        with open(label, 'r') as f:
            lines = f.readlines()
            jaywalker_coords = []  # jaywalker의 좌표를 저장할 리스트
            pedestrian_coords = []  # pedestrian의 좌표를 저장할 리스트
            for line in lines:
                if line.strip() and line[0] == '0':  # 클래스 ID가 '0'인 경우
                    jaywalker_coords.append(line.strip())  # 해당 라인을 좌표 리스트에 추가
                elif line.strip() and line[0] == '1':  # 클래스 ID가 '1'인 경우
                    pedestrian_coords.append(line.strip())  # 해당 라인을 좌표 리스트에 추가
            # 파일에 '0'과 '1'이 모두 있는 경우 jaywalker에 추가
            if jaywalker_coords and pedestrian_coords:
                jaywalker.append([label.split('/')[-1], jaywalker_coords + pedestrian_coords])
            # 파일에 '0'만 있는 경우에도 jaywalker에 추가
            elif jaywalker_coords:
                jaywalker.append([label.split('/')[-1], jaywalker_coords])
            # 파일에 '1'만 있는 경우 pedestrian에 추가
            elif pedestrian_coords:
                pedestrian.append([label.split('/')[-1], pedestrian_coords])

    jaywalker = sorted(jaywalker, key=lambda x: x[0])
    pedestrian = sorted(pedestrian, key=lambda x: x[0])


print('jaywalker',len(jaywalker))
print('pedestrian',len(pedestrian))
# print(len(glob.glob('./data/new_data/train/images/*.jpg')),len(glob.glob('./data/detection/labels/*.txt')))

jaywalker 10920
pedestrian 3490


# Augmentation 이미지 저장

In [4]:
import sys

sys.path.append(os.path.abspath('/content/drive/MyDrive/sejong/segmentation'))

In [5]:
from utils.dataset import myDataSet,Test_Detection
from utils.augmentation import augmentation_train, augmentation_test, detection_aug_train

In [50]:
jaywalker_img = [f"./data/new_data/train/images/{i[0].replace('.txt','.jpg')}" for i in jaywalker]
jaywalker_label = [f"./data/new_data/train/labels/{i[0]}" for i in jaywalker]

print(len(jaywalker_img))
print(len(jaywalker_label))

1560
1560


In [51]:
pedestrian_img = [f"./data/new_data/train/images/{i[0].replace('.txt','.jpg')}" for i in pedestrian]
pedestrian_label = [f"./data/new_data/train/labels/{i[0]}" for i in pedestrian]

print(len(pedestrian_img))
print(len(pedestrian_label))

3490
3490


In [44]:
print(1560+(1560*6))
print(3490+(3490*2))

10920
10470


## jaywalker augmentation 이미지

In [54]:
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

det_transforms = detection_aug_train()
train_img_path = sorted(jaywalker_img,key = lambda x: int(x.split('/')[-1].split('.')[0]))
train_label_path = sorted(jaywalker_label,key = lambda x: int(x.split('/')[-1].split('.')[0]))
num = len(glob.glob('./data/new_data/train/images/*.jpg'))
for i in range(6):
  for idx,(train_img,train_label) in enumerate(zip(train_img_path,train_label_path)):
    # with open(seg_label, 'r') as f:
    #     lines = f.readlines()
    overall_idx = i * len(train_img_path) + idx
    img = cv2.imread(train_img)
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img = det_transforms(img)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # print(f'{overall_idx}번째',train_img.split('/')[-1])
    # cv2_imshow(img)
  # print(lines)

    cv2.imwrite(f'./data/new_data/train/images/{overall_idx+num}.jpg',img)
    shutil.copy(train_label,f'./data/new_data/train/labels/{overall_idx+num}.txt')
    # print(f'/content/drive/MyDrive/sejong/ex_detection/original/train/images/{overall_idx+num}.jpg',img)
    # print(train_label,f'/content/drive/MyDrive/sejong/ex_detection/original/train/labels/{overall_idx+num}.txt')

In [57]:
print(len(glob.glob('./data/new_data/train/images/*.jpg')))
print(len(glob.glob('./data/new_data/train/labels/*.txt')))

14410
14410


## pedestrian augmentation 이미지

In [60]:
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

det_transforms = detection_aug_train()
train_img_path = sorted(pedestrian_img,key = lambda x: int(x.split('/')[-1].split('.')[0]))
train_label_path = sorted(pedestrian_label,key = lambda x: int(x.split('/')[-1].split('.')[0]))

num = len(glob.glob('./data/new_data/train/images/*.jpg'))

for i in range(2):
  for idx,(train_img,train_label) in enumerate(zip(train_img_path,train_label_path)):
    # with open(seg_label, 'r') as f:
    #     lines = f.readlines()
    overall_idx = i * len(train_img_path) + idx
    img = cv2.imread(train_img)
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img = det_transforms(img)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # print(f'{overall_idx}번째',train_img.split('/')[-1])
    # cv2_imshow(img)
  # print(lines)
    # print(train_label)
    cv2.imwrite(f'./data/new_data/train/images/{overall_idx+num}.jpg',img)
    shutil.copy(train_label,f'./data/new_data/train/labels/{overall_idx+num}.txt')

    # print(f'/content/drive/MyDrive/sejong/ex_detection/ori_seg/train/images/{overall_idx+num}.jpg',img)
    # print(f'/content/drive/MyDrive/sejong/ex_detection/ori_seg/train/labels/{overall_idx+num}.txt')